<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/N_spell_check%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='N')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000020,N,76,761,영업장에서,일반 고객을 대상,자동차 임대업
1,id_0000122,N,75,752,여행사에서,일반고객대상,해외여행알선
2,id_0000150,N,74,742,고객이원하는장소,고객요구에따라,건물내부청소
3,id_0000185,N,74,742,건물의 내부 외부에서,고객의 요청으로,"건물위생관리,청소 대행"
4,id_0000248,N,75,752,사무실에서,일반인 대상,국.내외 여행알선서비스
...,...,...,...,...,...,...,...
17696,id_0999708,N,74,742,공공장소 및 일반인의 의뢰로,공공장소에서,건축물청소
17697,id_0999715,N,75,752,여행사에서,일반고객대상으로,여행알선
17698,id_0999722,N,75,751,직업소개소에서,사업체요구에의해,인력공급
17699,id_0999890,N,76,762,가전제품 대여,NaN,가전제품 대여


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000020,N,76,761,영업장에서 일반 고객을 대상 자동차 임대업
1,id_0000122,N,75,752,여행사에서 일반고객대상 해외여행알선
2,id_0000150,N,74,742,고객이원하는장소 고객요구에따라 건물내부청소
3,id_0000185,N,74,742,"건물의 내부 외부에서 고객의 요청으로 건물위생관리,청소 대행"
4,id_0000248,N,75,752,사무실에서 일반인 대상 국.내외 여행알선서비스
...,...,...,...,...,...
17696,id_0999708,N,74,742,공공장소 및 일반인의 의뢰로 공공장소에서 건축물청소
17697,id_0999715,N,75,752,여행사에서 일반고객대상으로 여행알선
17698,id_0999722,N,75,751,직업소개소에서 사업체요구에의해 인력공급
17699,id_0999890,N,76,762,가전제품 대여 가전제품 대여


In [8]:
df['digit_3'].value_counts()

751    3712
759    3583
752    2605
762    1948
742    1866
763    1482
761     938
741     783
753     524
743     239
764      21
Name: digit_3, dtype: int64

## 맞춤법 교정

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-qxg38irp
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-qxg38irp
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=6f9f0ef4ab330fc42a938f844e05f074c66957e6b76dd252e4d2ad06c12d4136
  Stored in directory: /tmp/pip-ephem-wheel-cache-07ziu060/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_N_spellcheck.csv', index=False, encoding='utf-8-sig')